<a href="https://colab.research.google.com/github/ghommidhWassim/GNN-variants/blob/main/clusterGCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"
!pip install torchvision
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
2.6.0+cu124
12.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    

In [2]:
# Standard libraries
import numpy as np
from scipy import sparse
import seaborn as sns
import pandas as pd

# Plotting libraries
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib import cm
from IPython.display import Javascript  # Restrict height of output cell.

# PyTorch
import torch
import torch.nn.functional as F
from torch.nn import Linear
import torch.nn as nn
from torch_sparse import spmm
# import pyg_lib
import torch_sparse
from torch_geometric.utils import to_networkx

# PyTorch geometric
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import ClusterData, ClusterLoader
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.data import Data
from torch_geometric import seed_everything
from torch.profiler import profile, record_function, ProfilerActivity
import os.path as osp
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import GraphSAGE
from sklearn.metrics import accuracy_score
from transformers import AutoModelForCausalLM
from torch.profiler import profile, record_function, ProfilerActivity

In [15]:
def clean_gpu_memory():
    """Cleans GPU memory without fully resetting the CUDA context"""
    import gc
    gc.collect()  # Python garbage collection
    if torch.cuda.is_available():
        torch.cuda.empty_cache()  # PyTorch cache
        torch.cuda.reset_peak_memory_stats()  # Reset tracking
        print(f"Memory after cleanup: {torch.cuda.memory_allocated()/1024**2:.2f} MB")


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Initial allocated memory: {torch.cuda.memory_allocated() / (1024**2):.2f} MB")

def dataset_load():
  print(f"Using device: {device}")
  dataset = Planetoid(root='data/Planetoid', name='PubMed', transform=NormalizeFeatures())
  num_features = dataset.num_features
  num_classes = dataset.num_classes
  data = dataset[0].to(device)  # Get the first graph object.
  return num_features, data, num_classes, device,dataset

Initial allocated memory: 0.00 MB


In [4]:
num_features, data, num_classes, device, dataset = dataset_load()
print(f'Number of nodes:          {data.num_nodes}')
print(f'Number of edges:          {data.num_edges}')
print(f'Average node degree:      {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes:       {data.has_isolated_nodes()}')
print(f'Has self-loops:           {data.has_self_loops()}')
print(f'Is undirected:            {data.is_undirected()}')

Using device: cuda


Processing...
Done!


Number of nodes:          19717
Number of edges:          88648
Average node degree:      4.50
Number of training nodes: 60
Training node label rate: 0.003
Has isolated nodes:       False
Has self-loops:           False
Is undirected:            True


In [28]:
class GCN(torch.nn.Module):
  def __init__(self, hidden_channels):
    super().__init__()
    self.conv1 = GCNConv(num_features, hidden_channels)
    self.conv2 = GCNConv(hidden_channels, num_classes)

  def forward(self, x, edge_index):
    x = self.conv1(x, edge_index)
    x = x.relu()
    x = F.dropout(x, p=0.5, training=self.training)
    x = self.conv2(x, edge_index)
    return x

In [29]:
def train(data, mask):
  model.train()
  optimizer.zero_grad()  # Clear gradients.
  out = model(data.x, data.edge_index)  # Perform a single forward pass.
  loss = criterion(out[mask], data.y[mask])  # Compute the loss solely based on the training nodes.
  loss.backward()  # Derive gradients.
  optimizer.step()  # Update parameters based on gradients.
  return loss

def test(data, mask):
  model.eval()
  out = model(data.x.to(device), data.edge_index.to(device))
  pred = out.argmax(dim=1)
  correct = pred[mask.to(device)] == data.y[mask.to(device)]
  acc = int(correct.sum()) / int(mask.sum())
  return acc

In [30]:
data = data.cpu()
cluster_data = ClusterData(data, num_parts=128)
train_loader = ClusterLoader(cluster_data, batch_size=32, shuffle=True)
criterion = torch.nn.CrossEntropyLoss().to(device)

Computing METIS partitioning...
Done!


In [31]:
def train_batch(loader):
  model.train()
  i=1
  for sub_data in train_loader:  # Iterate over each mini-batch.
    sub_data = sub_data.to(device)
    batch_mem = sum(t.element_size() * t.nelement() for t in [sub_data.x, sub_data.edge_index, sub_data.y])
    print(f"Batch {i}: {batch_mem/1024**2:.2f} MB cluster data")
    #torch.cuda.reset_peak_memory_stats()

    out = model(sub_data.x, sub_data.edge_index)  # Perform a single forward pass.
    print(f"  + Forward pass: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")
    loss = criterion(out[sub_data.train_mask], sub_data.y[sub_data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    optimizer.zero_grad()  # Clear gradients.
    i=i+1

In [34]:

model = GCN(hidden_channels=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss().to(device)

display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
data=data.to(device)
for epoch in range(1, 101):
  loss = train_batch(train_loader)
  if epoch % 10 == 0:
    train_acc = test(data, data.train_mask)
    val_acc = test(data, data.val_mask)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}')

<IPython.core.display.Javascript object>

Batch 1: 9.67 MB cluster data
  + Forward pass: 110.08 MB
Batch 2: 9.72 MB cluster data
  + Forward pass: 110.40 MB
Batch 3: 9.70 MB cluster data
  + Forward pass: 110.40 MB
Batch 4: 9.68 MB cluster data
  + Forward pass: 110.40 MB
Batch 1: 9.76 MB cluster data
  + Forward pass: 110.40 MB
Batch 2: 9.64 MB cluster data
  + Forward pass: 110.40 MB
Batch 3: 9.73 MB cluster data
  + Forward pass: 110.55 MB
Batch 4: 9.65 MB cluster data
  + Forward pass: 110.55 MB
Batch 1: 9.70 MB cluster data
  + Forward pass: 110.55 MB
Batch 2: 9.71 MB cluster data
  + Forward pass: 110.55 MB
Batch 3: 9.69 MB cluster data
  + Forward pass: 110.55 MB
Batch 4: 9.68 MB cluster data
  + Forward pass: 110.55 MB
Batch 1: 9.66 MB cluster data
  + Forward pass: 110.55 MB
Batch 2: 9.74 MB cluster data
  + Forward pass: 110.55 MB
Batch 3: 9.71 MB cluster data
  + Forward pass: 110.55 MB
Batch 4: 9.67 MB cluster data
  + Forward pass: 110.55 MB
Batch 1: 9.72 MB cluster data
  + Forward pass: 110.55 MB
Batch 2: 9.66 

In [10]:
test_acc = test(data, data.test_mask)
test_acc

0.768

In [35]:
print(f"Allocated memory after tensor creation: {torch.cuda.memory_allocated() / (1024**2):.2f} MB")
print(f"Reserved memory after tensor creation: {torch.cuda.memory_reserved() / (1024**2):.2f} MB")
print(f"Peak allocated memory: {torch.cuda.max_memory_allocated() / (1024**2):.2f} MB")


Allocated memory after tensor creation: 96.75 MB
Reserved memory after tensor creation: 142.00 MB
Peak allocated memory: 114.60 MB


In [36]:
clean_gpu_memory()


Memory after cleanup: 96.75 MB


**another cluster GCN**

In [39]:
data.cpu()

cluster_data = ClusterData(data, num_parts=128)
train_loader = ClusterLoader(cluster_data, batch_size=32, shuffle=True)
# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(num_features, 128, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.to(device)
data = data.to(device)

# GPU memory profiler wrapper
def print_gpu_memory(msg=""):
    allocated = torch.cuda.memory_allocated() / (1024**2)
    reserved = torch.cuda.memory_reserved() / (1024**2)
    peak = torch.cuda.max_memory_allocated() / (1024**2)
    print(f"{msg}\nAllocated: {allocated:.2f} MB, Reserved: {reserved:.2f} MB, Peak: {peak:.2f} MB\n")

# Training loop
def train():
    model.train()
    total_loss = 0
    for batch in cluster_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = F.cross_entropy(out[batch.train_mask], batch.y[batch.train_mask])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print_gpu_memory("After training epoch")
    return total_loss / len(cluster_loader)

# Evaluation
def test(data):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        correct = pred[mask].eq(data.y[mask]).sum().item()
        accs.append(correct / mask.sum().item())
    return accs

# Run training
for epoch in range(1, 31):
    loss = train()
    accs = test(data)
    print(f"Epoch {epoch:02d}, Loss: {loss:.4f}, Train: {accs[0]:.4f}, Val: {accs[1]:.4f}, Test: {accs[2]:.4f}")


Computing METIS partitioning...
Done!


After training epoch
Allocated: 101.21 MB, Reserved: 144.00 MB, Peak: 120.93 MB

Epoch 01, Loss: 1.0910, Train: 0.7333, Val: 0.6080, Test: 0.6050
After training epoch
Allocated: 101.21 MB, Reserved: 252.00 MB, Peak: 225.77 MB

Epoch 02, Loss: 1.0222, Train: 0.9167, Val: 0.7240, Test: 0.6750
After training epoch
Allocated: 100.73 MB, Reserved: 252.00 MB, Peak: 225.77 MB

Epoch 03, Loss: 0.8751, Train: 0.8667, Val: 0.6840, Test: 0.6720
After training epoch
Allocated: 100.73 MB, Reserved: 252.00 MB, Peak: 225.77 MB

Epoch 04, Loss: 0.7848, Train: 0.8667, Val: 0.6960, Test: 0.6780
After training epoch
Allocated: 100.73 MB, Reserved: 252.00 MB, Peak: 225.77 MB

Epoch 05, Loss: 0.6821, Train: 0.9500, Val: 0.7780, Test: 0.7330
After training epoch
Allocated: 100.78 MB, Reserved: 252.00 MB, Peak: 225.77 MB

Epoch 06, Loss: 0.6063, Train: 0.9500, Val: 0.7560, Test: 0.7320
After training epoch
Allocated: 100.72 MB, Reserved: 252.00 MB, Peak: 225.77 MB

Epoch 07, Loss: 0.4703, Train: 0.9667, Val:

In [40]:
print(f"Allocated memory after tensor creation: {torch.cuda.memory_allocated() / (1024**2):.2f} MB")
print(f"Reserved memory after tensor creation: {torch.cuda.memory_reserved() / (1024**2):.2f} MB")
print(f"Peak allocated memory: {torch.cuda.max_memory_allocated() / (1024**2):.2f} MB")


Allocated memory after tensor creation: 97.64 MB
Reserved memory after tensor creation: 252.00 MB
Peak allocated memory: 225.77 MB


In [24]:
print(next(model.parameters()).device)  # Should say 'cuda'
print(data.x.device)  # Should say 'cuda'


cuda:0
cuda:0
